### Implementação da Camada Bronze

Esta etapa é responsável pela Ingestão e Catalogação dos dados brutos. Seu objetivo principal é capturar os dados da fonte original e persistí-los no Data Lake em formato otimizado, sem aplicar transformações de negócio, garantindo uma cópia fiel do histórico (raw data).

In [0]:
# Lê e carrega o dataset na camada bronze
df = spark.read.table("workspace.default.stroke_dataset")
df.write.format("delta").mode("overwrite").saveAsTable("mvpengenharia.bronze.stroke_dataset_bronze")

display(df.limit(10))

id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,null,0
30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,null,0
56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,null,0
32257,Female,47.0,0,0,Yes,Private,Urban,210.95,50.1,null,0
52800,Female,52.0,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0
41413,Female,75.0,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0
15266,Female,32.0,0,0,Yes,Private,Rural,77.67,32.3,smokes,0
28674,Female,74.0,1,0,Yes,Self-employed,Urban,205.84,54.6,never smoked,0


Os dados foram lidos de sua origem temporária **(workspace.default.stroke_dataset)** e gravados na tabela definitiva da camada Bronze **(mvpengenharia.bronze.stroke_dataset_bronze)**.

In [0]:
%sql
-- Query para identificar IDs duplicados
SELECT 
    id, 
    count(*) as repeticoes
FROM mvpengenharia.bronze.stroke_dataset_bronze
GROUP BY id
HAVING count(*) > 1
ORDER BY repeticoes DESC;

id,repeticoes


Foi executada uma consulta SQL para verificar a existência de registros duplicados através da coluna id. Esta etapa serve como um profiling inicial para orientar as estratégias de limpeza necessárias na próxima camada (Silver). Como o código não retornou nenhuma linha, fica claro que não há registros duplicados na tabela.

In [0]:
# Definindo o nome da tabela
table_name = "mvpengenharia.bronze.stroke_dataset_bronze"

# Comentário na Tabela
spark.sql(f"""
  COMMENT ON TABLE {table_name} IS 
  'Tabela da camada Bronze contendo dados brutos sobre ocorrências de AVC (Stroke), incluindo dados demográficos, clínicos e de estilo de vida dos pacientes.'
""")

DataFrame[]

In [0]:
# Definição das Colunas e Descrições
columns_descriptions = [
    ("id", "Identificador único do paciente"),
    ("gender", "Gênero do paciente (Male, Female, Other)"),
    ("age", "Idade do paciente"),
    ("hypertension", "Indicador de hipertensão (0: Não, 1: Sim)"),
    ("heart_disease", "Indicador de doença cardíaca (0: Não, 1: Sim)"),
    ("ever_married", "Indica se o paciente já foi casado (Yes/No)"),
    ("work_type", "Tipo de trabalho (ex: Private, Self-employed, Govt_job)"),
    ("Residence_type", "Tipo de residência (Rural, Urban)"),
    ("avg_glucose_level", "Nível médio de glicose no sangue"),
    ("bmi", "Índice de Massa Corporal (Body Mass Index)"),
    ("smoking_status", "Status de tabagismo (ex: never smoked, smokes)"),
    ("stroke", "Variável alvo: 1 se o paciente teve AVC, 0 caso contrário"),
]

# Loop para aplicar os comentários nas colunas
for col_name, comment in columns_descriptions:
    try:
        spark.sql(f"COMMENT ON COLUMN {table_name}.{col_name} IS '{comment}'")
        print(f"Comentário adicionado na coluna: {col_name}")
    except Exception as e:
        print(f"Erro ao comentar na coluna {col_name}: {e}")

Comentário adicionado na coluna: id
Comentário adicionado na coluna: gender
Comentário adicionado na coluna: age
Comentário adicionado na coluna: hypertension
Comentário adicionado na coluna: heart_disease
Comentário adicionado na coluna: ever_married
Comentário adicionado na coluna: work_type
Comentário adicionado na coluna: Residence_type
Comentário adicionado na coluna: avg_glucose_level
Comentário adicionado na coluna: bmi
Comentário adicionado na coluna: smoking_status
Comentário adicionado na coluna: stroke


Na célula acima foi implementada a documentação do Catálogo de Dados. Foram inseridos metadados descritivos na tabela e em cada uma de suas colunas. Isso assegura que o significado técnico e de negócio de cada atributo esteja disponível para qualquer usuário.